# Example of calculating atom specific property

Damn, I really want to know what the Fukui coefficient for particular atoms centers.

- Find some conformers
- Find the best conformer (based on energy)
- Calculate Fukui coefficients
- Map the coefficients to atoms


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import MolsToGridImage, MolToImage
from rdkit.Chem.MolStandardize import rdMolStandardize

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import jupyter as ppqm_jupyter

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)
show_progress = True

## Define a molecule you like

In [ ]:
smiles = "CCOC(=O)C1=C(C)N=C(C)/C(=C(\O)OCC)C1C"  # CHEMBL3189958
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

In [ ]:
xtb_options = {
    "scr": "./_tmp_directory_",  # Where should the calculations happen?
    "cmd": "xtb",  # What is the executable?
    "n_cores": 4,  # How many cores to use?
    "show_progress": show_progress,
}

In [ ]:
calc = ppqm.xtb.XtbCalculator(**xtb_options)

In [ ]:
calc

## Get a single conformer




In [ ]:
optimize_options = {
    "gfn": 2,
    "alpb": "h2o",
    "opt": None,
}

In [ ]:
# Generate conformers
molobj_conf = ppqm.tasks.generate_conformers(molobj)

In [ ]:
molobj_conf.GetNumConformers()

In [ ]:
# Optimize conformers
results = calc.calculate(molobj_conf, optimize_options)

In [ ]:
# Collect all the xTB energies
conformer_energies = [result["scc_energy"] for result in results]

In [ ]:
# Find the lowest energy
idx = np.argmin(conformer_energies)

In [ ]:
# Copy only the graph of the molecule, keeping the atom order
molobj_prime = ppqm.chembridge.copy_molobj(molobj_conf)  # Mol(mol, True)

# Get the coordinates of the best conformer
coord = results[idx]["coord"]

# Add conformer to molobj
ppqm.chembridge.molobj_add_conformer(molobj_prime, coord)

In [ ]:
ppqm_jupyter.show_molobj(molobj_prime)

## Calculate Fukui coefficients and map to atoms

In [ ]:
atoms = molobj_prime.GetAtoms()
atoms = list(atoms)
print(len(atoms))
print([atom.GetSymbol() for atom in atoms])

In [ ]:
# Calculate fukui coefficients
fukui_options = {
    "gfn": 2,
    "alpb": "h2o",
    "vfukui": None,
}

In [ ]:
# Results are returned as list(properties_dictionary), however we only have one conformer here.
# So note that the list only has one dictionary
fukui_results = calc.calculate(molobj_prime, fukui_options)

In [ ]:
fukui_results

In [ ]:
fukui_result = fukui_results[0]

In [ ]:
df = pd.DataFrame(fukui_result)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
fukui = fukui_result["f_minus"]

In [ ]:
# copy molobj to make a pretty picture
molobj_omega = ppqm.chembridge.copy_molobj(molobj_prime)

In [ ]:
for coef, atom in zip(fukui, molobj_omega.GetAtoms()):
    label = f"{coef:.2f}"
    atom.SetProp("atomNote", label)

In [ ]:
MolToImage(
    Chem.RemoveHs(molobj_omega),
    size=(500, 500),
)